# House Property Sales Analysis

## 1. Dataset

In this notebook, we are going to analyze House Property Sales data for 2007-2019 (Kaggle). The dataset contains information about the houses in an Australian city with 1,2,3,4,5 bedrooms.

The dataset includes the following variables:

**Datesold:**     The date when an owner sold the house to a buyer. <br>
**Postcode:**    4 digit postcode of the suburb where the owner sold the property <br>
**Price:**           Price for which the owner sold the property. <br>
**Bedrooms:**  Number of bedrooms. <br>


We are going to find the answers to questions like:
*  Which date corresponds to the highest number of sales?
*  Find out the postcode with the highest average price per sale? (Using Aggregate Functions)
*  Which year witnessed the lowest number of sales?
*  Use the window function to deduce the top six postcodes by year's price.

<br>
In this project, we'll use SQL in Jupyter Notebook to explore and analyze data from this database.<br>
The first line of the code imports pandas and sqlite3 libraries

In [1]:
import pandas as pd
import sqlite3

Next we connect the Jupyter notebook to the database file. <br>
The data is stored in the *property.db* file.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///property.db 

# Here we connect the Jupyter notebook to the database file.

Here we format the input as a table. <br><br>
**%%sql** will make it possible to run a SQL query with Jupyter

In [3]:
%%sql

select *
  from sqlite_master
 where type='table';

 * sqlite:///property.db
Done.


type,name,tbl_name,rootpage,sql
table,property,property,2,"CREATE TABLE ""property"" (""index"" INTEGER, ""datesold"" TEXT, ""postcode"" INTEGER, ""price"" INTEGER, ""propertyType"" TEXT, ""bedrooms"" INTEGER)"


In next step we'll look for the data by limiting the output to first 5 rows of the dataset to keep the output clean.<br>
We can see the *price of the houses*, *postcodes*, *property type* and *number of bedrooms* along with the *date* on which it was sold.

In [4]:
%%sql
SELECT *
FROM property
limit 5;

 * sqlite:///property.db
Done.


index,datesold,postcode,price,propertyType,bedrooms
0,2007-02-07,2607,525000,house,4
1,2007-02-27,2906,290000,house,3
2,2007-03-07,2905,328000,house,3
3,2007-03-09,2905,380000,house,4
4,2007-03-21,2906,310000,house,3


## 2.Finding the number of distinct types of property

In this section, we are going to extract the number of unique types of properties present in the table.

In [5]:
%%sql
select
            distinct propertyType
from property


 * sqlite:///property.db
Done.


propertyType
house
None
unit


## 3. Finding out the number of bedrooms

We can see there are 2 types of properties(remove the data with property type as 'None'). Houses have different number of bedrooms, so, we will look for the distinct number of bedrooms in the table

In [6]:
%%sql
select
            distinct bedrooms
from property
            order by bedrooms

 * sqlite:///property.db
Done.


bedrooms
1
2
3
4
5


## 4. Find out the distinct postcodes

We can see there are houses with 1,2,3,4,5 bedroom.
Next, to find out the number of postcodes given in the data, we will use the following SQL command

In [7]:
%%sql
select
            count(distinct postcode) as Number_of_post_codes
from property

 * sqlite:///property.db
Done.


Number_of_post_codes
27


## 5. Find out the minimum and the maximum price

From the above query we can see that the data given is for *27* postcodes.
Now that we have the number of postcodes, let's now find out the minimum and the maximum price of the house properties along with the postcode, datesold and property type. 

In [8]:
%%sql
select
            min(price) as Min_Price,
            postcode,
            propertyType,
            datesold
from property

 * sqlite:///property.db
Done.


Min_Price,postcode,propertyType,datesold
56500,2611,house,2015-04-24


In [9]:
%%sql
select
            max(price) as Max_price,
            postcode,
            propertyType,
            datesold
from property

 * sqlite:///property.db
Done.


Max_price,postcode,propertyType,datesold
8000000,2611,house,2015-11-02


## 6. Find the date with the highest property prices

In [10]:
%%sql
select
        datesold  as Date,
        sum(price) as Total_sales
from property
        group by datesold
        order by sum(price) desc
        limit 1

 * sqlite:///property.db
Done.


Date,Total_sales
2017-10-28,42421600


## 7. Find out the postcode with highest avgerage price per sale

Now we know that the minimum and maximum price are *56.5K* and *8M* respectively and both the prices are at same postcode i.e. *2611* in year *2015*.

Let's look for postcode with the highest average sale price among all postcodes

In [11]:
%%sql
with avg_sales as(
            select
                    distinct postcode,
                    cast(avg(price) as int) as avg_price_per_sale
            from property
                group by postcode
       )
select 
            postcode,
            max(avg_price_per_sale) as highest_avg_price_per_sale
from avg_sales
           
        

 * sqlite:///property.db
Done.


postcode,highest_avg_price_per_sale
2618,1081111


## 8. Find the date with most frequent sales

Postcode *2618* has the highest average sales price *1.08M*.

Let's write a query to look for the date with most sales count and the sum of the price on that particular day.

In [12]:
%%sql
select 
            strftime('%Y-%m-%d',datesold) as year,
            count(*) as sales_count,
            sum(price) as sum_of_price
from property
        group by datesold
        order by sales_count desc 
        limit 5;

 * sqlite:///property.db
Done.


year,sales_count,sum_of_price
2017-10-28,50,42421600
2017-11-18,39,34661000
2018-03-24,38,32819000
2017-11-11,37,31425200
2017-04-08,37,31556000


## 9. Find the postcode with most sales by count

50 properties were sold worth *42.42M* on **28th Oct 2017** i.e. the most sales count in a day.<br>
Let's find out the postcode with most sales count from *2007* to *2019*. <br>

In [13]:
%%sql
select 
            postcode,
            count(*) as sales_count
from property
            group by postcode
            order by sales_count desc
            limit 5;

 * sqlite:///property.db
Done.


postcode,sales_count
2615,3564
2602,2599
2913,2556
2905,2259
2617,2114


## 10. Find top the 3 postcodes with highest sales price each year

Postcode *2615* is on the top in terms of sales count i.e. 3564 properties sold from the year 2007 to 2019.<br>
Now, we will look for the top 3 postcodes by the highest sum of price every year.

In [14]:
%%sql
with top_postcodes_by_year as(
        select
                strftime('%Y',datesold) as year,
                postcode,
                sum(price),
                dense_rank() over(partition by strftime('%Y',datesold) order by sum(price) desc) as rank_
        from property
            group by 1,2
)
select 
            * 
from top_postcodes_by_year
            where rank_ <=3

 * sqlite:///property.db
Done.


year,postcode,sum(price),rank_
2007,2602,11225500,1
2007,2905,6415500,2
2007,2906,5696500,3
2008,2611,32017750,1
2008,2602,29482950,2
2008,2906,27925000,3
2009,2905,72295975,1
2009,2615,67330345,2
2009,2602,65746250,3
2010,2602,96522423,1


## 11. Find the % change in average price of a property compared to previous year.

We have used Common Table Expression(CTE) in the above query for better readibility, used dense_rank() function to rank the postcodes on the basis of year and sum of property prices in that area and then filtered the top 3 postcodes in each year. <br>
<br> 
Now, we will find out the % change in average price of the properties using lag() function and use coalesce() function to handle NULL values.

### The following query is to find out the % change in the average price for properties with 5 bedrooms compared to the previous year.

In [15]:
%%sql
with cte as(
select
            distinct strftime('%Y',datesold) as year,
            cast(avg(price) as int) as avg_price
from property
        where bedrooms = 1
            group by 1
),
cte_2 as(
select
            *,
            coalesce(lag(avg_price,1) over(order by year),0) as prev_year_price
            
    from cte)
select
        *,
        coalesce((avg_price - prev_year_price)*100/prev_year_price , 0)  as  pct_change
from cte_2

 * sqlite:///property.db
Done.


year,avg_price,prev_year_price,pct_change
2007,278900,0,0
2008,310266,278900,11
2009,300340,310266,-3
2010,336753,300340,12
2011,357378,336753,6
2012,351710,357378,-1
2013,340007,351710,-3
2014,363718,340007,6
2015,324337,363718,-10
2016,322143,324337,0


### The following query is to find out the % change in the average price for properties with 2 bedrooms compared to the previous year.

In [16]:
%%sql
with cte as(
select
            distinct strftime('%Y',datesold) as year,
            cast(avg(price) as int) as avg_price
from property
            where bedrooms = 2
            group by 1
),
cte_2 as(
select
            *,
            coalesce(lag(avg_price,1) over(order by year),0) as prev_year_price
            
    from cte)
select
        *,
        coalesce((avg_price - prev_year_price)*100/prev_year_price , 0)  as  pct_change
from cte_2

 * sqlite:///property.db
Done.


year,avg_price,prev_year_price,pct_change
2007,369996,0,0
2008,396381,369996,7
2009,404590,396381,2
2010,451798,404590,11
2011,437060,451798,-3
2012,438873,437060,0
2013,430299,438873,-1
2014,442868,430299,2
2015,446744,442868,0
2016,455457,446744,1


### The following query is to find out the % change in the average price for properties with 3 bedrooms compared to the previous year.

In [17]:
%%sql
with cte as(
select
            distinct strftime('%Y',datesold) as year,
            cast(avg(price) as int) as avg_price
from property
            where bedrooms = 3
            group by 1
),
cte_2 as(
select
            *,
            coalesce(lag(avg_price,1) over(order by year),0) as prev_year_price
            
    from cte)
select
        *,
        coalesce((avg_price - prev_year_price)*100/prev_year_price , 0)  as  pct_change
from cte_2

 * sqlite:///property.db
Done.


year,avg_price,prev_year_price,pct_change
2007,459947,0,0
2008,438253,459947,-4
2009,448192,438253,2
2010,503139,448192,12
2011,511957,503139,1
2012,502150,511957,-1
2013,505460,502150,0
2014,523178,505460,3
2015,555810,523178,6
2016,585767,555810,5


### The following query is to find out the % change in the average price for properties with 4 bedrooms compared to the previous year.

In [18]:
%%sql
with cte as(
select
            distinct strftime('%Y',datesold) as year,
            cast(avg(price) as int) as avg_price
from property
            where bedrooms = 4
            group by 1
),
cte_2 as(
select
            *,
            coalesce(lag(avg_price,1) over(order by year),0) as prev_year_price
            
    from cte)
select
        *,
        coalesce((avg_price - prev_year_price)*100/prev_year_price , 0)  as  pct_change
from cte_2

 * sqlite:///property.db
Done.


year,avg_price,prev_year_price,pct_change
2007,628479,0,0
2008,559718,628479,-10
2009,573333,559718,2
2010,627649,573333,9
2011,651132,627649,3
2012,627624,651132,-3
2013,632656,627624,0
2014,673591,632656,6
2015,728415,673591,8
2016,745750,728415,2


### The following query is to find out the % change in the average price for properties with 5 bedrooms compared to the previous year.

In [19]:
%%sql
with cte as(
select
            distinct strftime('%Y',datesold) as year,
            cast(avg(price) as int) as avg_price
from property
            where bedrooms = 5
            group by 1
),
cte_2 as(
select
            *,
            coalesce(lag(avg_price,1) over(order by year),0) as prev_year_price
            
    from cte)
select
        *,
        coalesce((avg_price - prev_year_price)*100/prev_year_price , 0)  as  pct_change
from cte_2

 * sqlite:///property.db
Done.


year,avg_price,prev_year_price,pct_change
2007,786300,0,0
2008,792990,786300,0
2009,714441,792990,-9
2010,856062,714441,19
2011,847338,856062,-1
2012,825651,847338,-2
2013,761837,825651,-7
2014,858843,761837,12
2015,928442,858843,8
2016,963874,928442,3
